# Simple User-Based Collaborative Filtering

This notebook demonstrates a simple implementation of user-based collaborative filtering using the MovieLens dataset.

The process includes:
1. Loading the MovieLens dataset
2. Training a user-based KNN model
3. Finding similar users to a test subject
4. Generating personalized recommendations based on similar users' ratings

In [ ]:
# Import required libraries
from recsys.MovieLens import MovieLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter

## Setup and Data Loading

We'll use user ID 85 as our test subject and set k=10 for the number of similar users to consider.

In [ ]:
# Set parameters
testSubject = '85'
k = 10

# Load the MovieLens dataset
ml, data, ratings = MovieLens.load()

# Build the training set
trainSet = data.build_full_trainset()

## Train the Model

We'll train a user-based KNN model using cosine similarity.

In [ ]:
# Configure and train the model
sim_options = {
    'name': 'cosine',
    'user_based': True
}

model = KNNBasic(sim_options=sim_options)
model.fit(trainSet)
simsMatrix = model.compute_similarities()

## Generate Recommendations

For our test user, we'll:
1. Find their k most similar users
2. Get items rated by similar users
3. Generate recommendations based on weighted ratings from similar users

In [ ]:
# Get the test user's inner ID
testUserInnerID = trainSet.to_inner_uid(testSubject)

# Find similar users
similarityRow = simsMatrix[testUserInnerID]

similarUsers = []
for innerID, score in enumerate(similarityRow):
    if (innerID != testUserInnerID):
        similarUsers.append((innerID, score))

kNeighbors = heapq.nlargest(k, similarUsers, key=lambda t: t[1])

# Get items rated by similar users
candidates = defaultdict(float)
for similarUser in kNeighbors:
    innerID = similarUser[0]
    userSimilarityScore = similarUser[1]
    theirRatings = trainSet.ur[innerID]
    for rating in theirRatings:
        candidates[rating[0]] += (rating[1] / 5.0) * userSimilarityScore

# Track items the user has already seen
watched = {}
for itemID, rating in trainSet.ur[testUserInnerID]:
    watched[itemID] = 1

# Get and display top recommendations
print("Top 10 recommendations for user", testSubject, ":\n")
pos = 0
for itemID, ratingSum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        movieID = trainSet.to_raw_iid(itemID)
        print(ml.getMovieName(int(movieID)), ratingSum)
        pos += 1
        if (pos > 10):
            break